In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',0)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
import gc

In [ ]:
prods = pd.read_csv('../data/maestro_producto.csv',index_col=0)
cnslt = pd.read_csv('../data/maestro_consultora.csv',index_col=0)
camps = pd.read_csv('../data/campana_consultora.csv',index_col=0).sort_values(by=['IdConsultora','campana']).reset_index(drop=True)
hists = pd.read_csv('../data/dtt_fvta_cl.csv').sort_values(by=['idconsultora','campana']).reset_index(drop=True)
submt = pd.read_csv('../data/predict_submission.csv')

In [ ]:
def weighted_mean(x):
    weights = np.flip([(1/(i+2)) for i in range(len(x))])
    return np.average(x,weights=weights)

In [ ]:
# Simple weighted mean approach
from sklearn.metrics import roc_auc_score

lst = camps['IdConsultora'].isin(camps[camps['campana']<201906]['IdConsultora'])

res = pd.merge(camps[(camps['campana']==201906)][['IdConsultora','Flagpasopedido']],
         camps[(camps['campana']!=201906)].groupby('IdConsultora').agg({'Flagpasopedido':weighted_mean}).reset_index(),
         on = 'IdConsultora', how = 'left').iloc[:,-1].fillna(1).values
roc_auc_score(camps[camps['campana']==201906]['Flagpasopedido'].values, res)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(camps[camps['campana']==201906]['Flagpasopedido'].values, res)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
submit = pd.merge(submt['idconsultora'],
         camps.rename(columns={'IdConsultora':'idconsultora'}).groupby('idconsultora').agg({'Flagpasopedido':weighted_mean}).reset_index(),
         on = 'idconsultora', how = 'left')

In [ ]:
submit.head()

In [ ]:
sns.distplot(submit['Flagpasopedido'])

In [ ]:
submit.to_csv('../subs/submission.csv',index=None,encoding='utf-8',sep=',')
